In [84]:
import importlib
import xarray as xr
import numpy as np
import sys

from CASutils import calendar_utils as cal
from CASutils import grid_utils as grid

importlib.reload(cal)
importlib.reload(grid)

<module 'CASutils.grid_utils' from '/home/islas/python/CASanalysis/CASutils/grid_utils.py'>

In [72]:
pathout="/project/cas/islas/python_savs/snowpaper/DATA_SORT/3cities/"

In [11]:
cityname=['Saskatoon','Toronto','Siderovsk']
citylon=[253.330, 280.617, 82.3139]
citylat=[52.1579, 43.6532, 66.5973]

In [2]:
ystart=1979 ; yend=2014 ; nyears=yend-ystart+1

In [3]:
dat1970 = xr.open_dataset("/project/haggis/obs/BEST_daily/Complete_TAVG_Daily_LatLong1_1970.nc")
dat1980 = xr.open_dataset("/project/haggis/obs/BEST_daily/Complete_TAVG_Daily_LatLong1_1980.nc")
dat1990 = xr.open_dataset("/project/haggis/obs/BEST_daily/Complete_TAVG_Daily_LatLong1_1990.nc")
dat2000 = xr.open_dataset("/project/haggis/obs/BEST_daily/Complete_TAVG_Daily_LatLong1_2000.nc")
dat2010 = xr.open_dataset("/project/haggis/obs/BEST_daily/Complete_TAVG_Daily_LatLong1_2010.nc")

In [6]:
# convert the date_number into a useful time axis
time = cal.fracofyear2date(np.array(dat1970.date_number))
dat1970 = dat1970.assign_coords(time=time)

time = cal.fracofyear2date(np.array(dat1980.date_number))
dat1980 = dat1980.assign_coords(time=time)

time = cal.fracofyear2date(np.array(dat1990.date_number))
dat1990 = dat1990.assign_coords(time=time)

time = cal.fracofyear2date(np.array(dat2000.date_number))
dat2000 = dat2000.assign_coords(time=time)

time = cal.fracofyear2date(np.array(dat2010.date_number))
dat2010 = dat2010.assign_coords(time=time)

In [46]:
# concatenate
best = xr.concat([dat1970, dat1980, dat1990, dat2000, dat2010], "time", data_vars=["temperature"])

In [47]:
# remove Feb 29ths
best = best.sel(time=~((best.time.dt.month==2) & (best.time.dt.day == 29)))
best.time.encoding['calendar'] = "noleap"

In [48]:
# grab out 1979-2014
best = best.sel(time=slice("1979-01-01","2014-12-31"))

In [49]:
# repeat the climatology, assign appropriate coords and add to anomalies
#clim = np.repeat( np.array(best.climatology), nyears, axis=0)
clim = np.tile(best.climatology, (nyears, 1,1))
clim_xr = xr.DataArray(clim, dims = best.temperature.dims, coords = best.temperature.coords)
best['temperature'] = best['temperature']+clim_xr

In [85]:
best_flipped = grid.fliplon_neg2pos(best,'longitude')

In [86]:
besttrefht = xr.DataArray(np.zeros([best.time.size,3]), coords=[best.time, cityname], dims=['time','city'], name='best')
for icity in np.arange(0,len(cityname),1):
    besttrefht[:,icity] = best_flipped.temperature.sel(longitude=citylon[icity], latitude=citylat[icity], method="nearest")

In [88]:
besttrefht.to_netcdf(path=pathout+"BEST_TREFHT.nc")

In [90]:
print(besttrefht.time)

<xarray.DataArray 'time' (time: 13140)>
array(['1979-01-01T12:00:00.000000000', '1979-01-02T12:00:00.000000000',
       '1979-01-03T12:00:00.000000000', ..., '2014-12-29T12:00:00.000000000',
       '2014-12-30T12:00:00.000000000', '2014-12-31T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01T12:00:00 ... 2014-12-31T12:00:00
